In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
import json
with open('C:\\Users\\javaughn\\OneDrive - HORNE LLP\\Desktop\\yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['Client-id', 'api-key'])

In [3]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5)
yelp_api

In [4]:
# Set out API call parameters and filename before the first call
LOCATION = 'Memphis, TN, 38016'
TERM = 'Barbecue'

In [5]:
# Specifying JSON_FILE filename (can include a folder)
# Include the search terms in the filename
JSON_FILE = f"Data/results_in_progress_Barbecue.json"
JSON_FILE

'Data/results_in_progress_Barbecue.json'

In [6]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_Barbecue.json not found. Saving empty list to file.


In [7]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [8]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results+1)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [9]:
## How many results total?
total_results = results['total']
total_results

540

In [10]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [11]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

27

In [12]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [13]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/27 [00:00<?, ?it/s]

In [14]:
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results+1)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
#     display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)

  0%|          | 0/27 [00:00<?, ?it/s]

In [15]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,wDkU6d67eyONe8Y5DN5q1w,victory-lane-bbq-lakeland,Victory Lane BBQ,https://s3-media4.fl.yelpcdn.com/bphoto/GvTfrV...,False,https://www.yelp.com/biz/victory-lane-bbq-lake...,6,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.5,"{'latitude': 35.21336801575167, 'longitude': -...","[pickup, delivery]","{'address1': '3049 Davies Plantation Rd', 'add...",+19012028888,(901) 202-8888,4098.090119,NaN
1,Uqy3cJHMfc4crc4gpP8ugw,germantown-commissary-germantown,Germantown Commissary,https://s3-media2.fl.yelpcdn.com/bphoto/ovoruf...,False,https://www.yelp.com/biz/germantown-commissary...,1023,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.0,"{'latitude': 35.08646068169633, 'longitude': -...",[delivery],"{'address1': '2290 S Germantown Rd', 'address2...",+19017545540,(901) 754-5540,11152.323116,$$
2,w10k316Q5Aa7OXqypdjwsQ,ferguson-s-bbq-memphis,Ferguson’s BBQ,https://s3-media2.fl.yelpcdn.com/bphoto/ef9eUh...,False,https://www.yelp.com/biz/ferguson-s-bbq-memphi...,20,"[{'alias': 'bbq', 'title': 'Barbeque'}]",4.5,"{'latitude': 35.201531, 'longitude': -89.835702}","[pickup, delivery]","{'address1': '2770 Whitten Rd', 'address2': ''...",+19013215490,(901) 321-5490,6786.809153,NaN
3,7O0X85RPLM7Y5n8569-_Bg,corkys-bar-b-q-memphis,Corky's Bar-B-Q,https://s3-media1.fl.yelpcdn.com/bphoto/YdXDT0...,False,https://www.yelp.com/biz/corkys-bar-b-q-memphi...,209,"[{'alias': 'bbq', 'title': 'Barbeque'}]",3.5,"{'latitude': 35.1724870130312, 'longitude': -8...","[pickup, delivery]","{'address1': '1740 N Germantown Rd', 'address2...",+19017371988,(901) 737-1988,2575.668605,$$
4,58v51swfxBVywZBcGq65bg,tys-smokehouse-bartlett,Ty's Smokehouse,https://s3-media1.fl.yelpcdn.com/bphoto/gGifvQ...,False,https://www.yelp.com/biz/tys-smokehouse-bartle...,92,"[{'alias': 'smokehouse', 'title': 'Smokehouse'}]",4.0,"{'latitude': 35.204716872216096, 'longitude': ...","[pickup, delivery]","{'address1': '7174 Stage Rd', 'address2': 'Ste...",+19013848111,(901) 384-8111,5981.840989,$


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
534,FujfxdOnpZRGqVIIBDjDZQ,wendys-collierville,Wendy's,https://s3-media3.fl.yelpcdn.com/bphoto/NS3xMp...,False,https://www.yelp.com/biz/wendys-collierville?a...,43,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",1.5,"{'latitude': 35.04637, 'longitude': -89.68607}",[delivery],"{'address1': '714 W Poplar Ave', 'address2': '...",+19018546037,(901) 854-6037,16530.433193,$
535,fQ1X272mVxgxzQGfWUT13w,wendys-memphis-27,Wendy's,https://s3-media1.fl.yelpcdn.com/bphoto/UdkMfY...,False,https://www.yelp.com/biz/wendys-memphis-27?adj...,11,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",2.0,"{'latitude': 35.04907, 'longitude': -89.79892}",[delivery],"{'address1': '7928 Winchester Road', 'address2...",+19017514060,(901) 751-4060,14856.398767,$
536,HkVFe1agKAYKmYf2rlQcGQ,wendys-memphis-30,Wendy's,,False,https://www.yelp.com/biz/wendys-memphis-30?adj...,6,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",1.0,"{'latitude': 35.0503311, 'longitude': -89.8544...",[delivery],"{'address1': '6260 Winchester Rd', 'address2':...",+19017951466,(901) 795-1466,16535.763650,$
537,Bp7odxIuzVSIPpiU059BLA,arbys-memphis,Arby's,https://s3-media1.fl.yelpcdn.com/bphoto/NNZ0jS...,False,https://www.yelp.com/biz/arbys-memphis?adjust_...,13,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",3.0,"{'latitude': 35.0499611, 'longitude': -89.8057...","[pickup, delivery]","{'address1': '7734 Winchester Rd', 'address2':...",+19017537169,(901) 753-7169,14897.729544,$
538,1LnBltBfcjHsNi1TP2I42Q,chick-fil-a-memphis-2,Chick-fil-A,https://s3-media1.fl.yelpcdn.com/bphoto/hr_En8...,False,https://www.yelp.com/biz/chick-fil-a-memphis-2...,47,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",3.0,"{'latitude': 35.0494834, 'longitude': -89.8275...",[],"{'address1': '7072 Winchester Rd', 'address2':...",+19017371778,(901) 737-1778,15552.811301,$


In [16]:
# check for duplicate IDs
final_df.duplicated(subset='id').sum()

1

In [17]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_Barbecue.csv.gz', compression='gzip',index=False)